In [ ]:
import os
import struct
import sys
import numpy as np
import shutil
import requests
from timeit import default_timer as timer
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import FileLocationManager
from utilities.utilities_affine import DATA_PATH

In [ ]:
animal = 'DK39'
fileLocationManager = FileLocationManager(animal)
sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
src_url_id = 182 # DK52 this needs to be turned into a variable or looked up somehow
# Get src points data
df = sqlController.get_point_dataframe(src_url_id)
df = df[df['Layer'] == 'premotor']

In [ ]:
df.head(2)

In [ ]:
records = df[['X', 'Y', 'Section']].to_records(index=False)
result = list(records)

In [ ]:
for x,y,z in records:
    print(x,y,z)